In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [2]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [3]:
data_loc = '/gpfs/slac/atlas/fs1/d/rafaeltl/public/ML/L1RNN/datasets_2020_ff/'

In [4]:
file_str = 'Jan06_FlavFix_smear_1_std_xtd_zst.h5'

In [5]:
f5 = h5py.File(data_loc+file_str, 'r')

In [6]:
x_train = np.array( f5['x_train'] )
y_train = to_categorical ( np.array( f5['y_train'] ) )
w_train = np.array( f5['w_train'] )

In [ ]:
# import tensorflow.compat.v2 as tf
# tf.enable_v2_behavior()

In [7]:
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, LSTM, Masking, Input, GRU, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

In [8]:
from tensorflow.keras import regularizers

In [9]:
import qkeras

In [10]:
def lstmmodel(max_len, n_var, rec_units, ndense=[10], rec_layer='LSTM'):
    
    x = track_inputs = Input(shape=(max_len, n_var,))
    
    x = qkeras.QLSTM(
                    rec_units,
                    activation='quantized_tanh(4)',
                    recurrent_activation='quantized_relu(4,0,1)',
                    kernel_quantizer='stochastic_ternary("auto")',
                    recurrent_quantizer='quantized_bits(2,1,1,alpha=1.0)',
                    bias_quantizer='quantized_bits(4,0,1)')(x)

    
    for ind,nd in enumerate(ndense):
        x = qkeras.QDense(nd, 
                                kernel_quantizer="quantized_bits(4,0,1)",
                                bias_quantizer='quantized_bits(4,0,1)',
                                name=f'dense_{ind}' )(x)
        x = qkeras.QActivation('quantized_relu(4,0,1)')(x)
    
    output = qkeras.QDense(3, 
                            kernel_quantizer="quantized_bits(4,0,1)",
                            bias_quantizer='quantized_bits(4,0,1)', 
                           name = 'output_sigmoid')(x)
    
    output = qkeras.QActivation("quantized_bits(20, 5)")(output)
    
    output = Activation('softmax')(output)
    
    model = Model(inputs=track_inputs, outputs=output)
    
    d_layers = ''.join([ str(dl) for dl in ndense ])
#     mname = f'rnn_{rec_layer}_{rec_units}_{d_layers}_nomask'
#     mname = f'rnn_{rec_layer}_{rec_units}_{d_layers}_nomask_LSTMKIvs' #LSTM kernel initializer variance scaling
#     mname = f'rnn_{rec_layer}_{rec_units}_{d_layers}_nomask_LSTMKIgn' #LSTM kernel initializer glorot normal
#     mname = f'rnn_{rec_layer}_{rec_units}_{d_layers}_nomask_LSTMKIvs_KRl10.001l20.0001' #LSTM kernel regularizer
#     mname = f'rnn_{rec_layer}_{rec_units}_{d_layers}_nomask_LSTMKIvs_BRl111l21' #LSTM bias regularizer
#     mname = f'rnn_{rec_layer}_{rec_units}_{d_layers}_nomask_LSTMKIvs_ARl11l21' #LSTM bias regularizer
#     mname = f'rnn_{rec_layer}_{rec_units}_{d_layers}_nomask_LSTMKIvsReLu' #LSTM kernel regularizer

    mname = f'Qrnn_{rec_layer}_{rec_units}_{d_layers}' #LSTM kernel regularizer
    
    return model, mname

In [11]:
# model, model_name = lstmmodel(15, 6, 50, [10], rec_layer='GRU')
model, model_name = lstmmodel(15, 6, 16, [80], rec_layer='LSTM')

In [12]:
model.summary()
print(model_name)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15, 6)]           0         
_________________________________________________________________
qlstm (QLSTM)                (None, 16)                1472      
_________________________________________________________________
dense_0 (QDense)             (None, 80)                1360      
_________________________________________________________________
q_activation (QActivation)   (None, 80)                0         
_________________________________________________________________
output_sigmoid (QDense)      (None, 3)                 243       
_________________________________________________________________
q_activation_1 (QActivation) (None, 3)                 0         
_________________________________________________________________
activation (Activation)      (None, 3)                

In [13]:
model_json = model.to_json()
with open(f'keras/model_{model_name}_arch.json', "w") as json_file:
    json_file.write(model_json)

In [14]:
adam = Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])

In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [16]:
model_output = f'keras/model_{model_name}_weights.h5'

In [17]:
train = True

In [18]:
if train:
    history = model.fit( x_train , y_train,
            batch_size=2**14,
            epochs=150,
            validation_split=0.1,
            shuffle = True,
            sample_weight= w_train,
            callbacks = [
                EarlyStopping(verbose=True, patience=150, monitor='val_accuracy'),
                ModelCheckpoint(model_output, monitor='val_accuracy', verbose=True, save_best_only=True)
                ],
            verbose=True
            )
    
model.load_weights(model_output)

Epoch 1/150
39/39 [==============================] - ETA: 0s - loss: 1.0052 - accuracy: 0.4989
Epoch 00001: val_accuracy improved from -inf to 0.53963, saving model to keras/model_Qrnn_LSTM_16_80_weights.h5
39/39 [==============================] - 12s 313ms/step - loss: 1.0052 - accuracy: 0.4989 - val_loss: 0.9619 - val_accuracy: 0.5396
Epoch 2/150
39/39 [==============================] - ETA: 0s - loss: 0.9345 - accuracy: 0.5692
Epoch 00002: val_accuracy improved from 0.53963 to 0.60483, saving model to keras/model_Qrnn_LSTM_16_80_weights.h5
39/39 [==============================] - 8s 199ms/step - loss: 0.9345 - accuracy: 0.5692 - val_loss: 0.8973 - val_accuracy: 0.6048
Epoch 3/150
39/39 [==============================] - ETA: 0s - loss: 0.8856 - accuracy: 0.6186
Epoch 00003: val_accuracy improved from 0.60483 to 0.63715, saving model to keras/model_Qrnn_LSTM_16_80_weights.h5
39/39 [==============================] - 8s 195ms/step - loss: 0.8856 - accuracy: 0.6186 - val_loss: 0.8641 - 

In [ ]:
model.summary()

In [ ]:
x_test = np.array( f5['x_test'] )
y_test = to_categorical ( np.array( f5['y_test'] ) )

In [ ]:
import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [ ]:
import importlib

In [ ]:
importlib.reload(plotting)

In [ ]:
pred_test = model.predict(x_test, batch_size=2**10)

In [ ]:
print("Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(pred_test, axis=1))))

In [ ]:
pb_b = pred_test[:,0] [y_test[:,0] == 1]
pc_b = pred_test[:,1] [y_test[:,0] == 1]
pl_b = pred_test[:,2] [y_test[:,0] == 1]
    
pc_c = pred_test[:,1] [y_test[:,1] == 1]
pb_c = pred_test[:,0] [y_test[:,1] == 1]
    
pl_l = pred_test[:,2] [y_test[:,2] == 1]
pb_l = pred_test[:,0] [y_test[:,2] == 1]

plt.Figure()

plt.hist( pb_b/(pb_b+pl_b), range=(0,1), bins=1000, histtype='step' )
plt.hist( pb_l/(pb_l+pl_l), range=(0,1), bins=1000, histtype='step' )

plt.show()


plt.Figure()

plt.hist( pb_b/(pb_b+pc_b), range=(0,1), bins=1000, histtype='step' )
plt.hist( pb_c/(pb_c+pc_c), range=(0,1), bins=1000, histtype='step' )

plt.show()

In [ ]:
plt.figure(figsize=(9,9))
_ = plotting.makeRoc(y_test, pred_test)

In [ ]:
for layer in model.layers:
    print(layer.name)
#     plt.Figure()
    
    this_wgts = layer.get_weights()
#     if len(this_wgts) < 1: continue
    print(layer.get_config())
    
    for wgt in this_wgts:
        print(wgt)
        print()
#     max_wgts = np.max(this_wgts)
#     min_wgts = np.min(this_wgts)
#     plt.hist(this_wgts, bins=100, range=(min_wgts, max_wgts))
#     plt.xlabel(f'{layer.name}')
#     plt.show